In [1]:
from pathlib import Path
p = Path("exemples/dummyRepo/").absolute()
from statisfactory import Session
sess = Session(root_folder=p)

2022-07-14 09:24:28,694 [statisfactory.IO.artifacts.backend] : DEBUG : Registering 's3' backend
2022-07-14 09:24:28,695 [statisfactory.IO.artifacts.backend] : DEBUG : Registering '' backend
2022-07-14 09:24:28,696 [statisfactory.IO.artifacts.backend] : DEBUG : Registering 'lakefs' backend
2022-07-14 09:24:28,701 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'csv' interactor
2022-07-14 09:24:28,702 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'xslx' interactor
2022-07-14 09:24:28,703 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'pickle' interactor
2022-07-14 09:24:28,705 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'odbc' interactor
2022-07-14 09:24:28,706 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'datapane' interactor
2022-07-14 09:24:28,707 [statisfactory.IO.artifacts.artifact_interactor] : DEBUG : registering 'binary' interactor
2022-07-14 09:24:28,708 [

In [34]:
from typing import List, Dict
import json
from pydantic.dataclasses import dataclass
from pydantic.json import pydantic_encoder
from statisfactory.operator.pipeline.solver import DAGSolver



@dataclass
class Manifest():
    """
    Represent a minifest : a description of a stati project
    """
        
    @dataclass
    class Craft():
        module: str
        name: str

    
    pipelines: Dict[str, List[List[Craft]]]

    def json_dumps(self):
        return json.dumps(manifest, default=pydantic_encoder, indent=2)


pipelines = {}
for pipeline_name, pipeline in sess.pipelines_definitions.items():
    steps = []
    for batch in DAGSolver(pipeline.crafts):
        batch = [{'module': craft.__module__, 'name':craft.__name__} for craft in batch]
        steps.append(batch)
    pipelines[pipeline_name] = steps

manifest = Manifest(pipelines=pipelines)


In [40]:
from statisfactory.operator.annotations import AnnotationKind

In [41]:
[anno.name for anno in craft.input_annotations if anno.kind in (AnnotationKind.ARTEFACT, AnnotationKind.VOLATILE)]\

inputs = {'volatiles':[], 'artifacts':[]}
for annotation in craft.input_annotations:
    if annotation.kind == AnnotationKind.VOLATILE:
        inputs['volatiles'].append(annotation.name)
    elif annotation.kind == AnnotationKind.ARTEFACT:
        inputs['artifacts'].append(annotation.name)


outputs = {'volatiles':[], 'artifacts':[]}
for annotation in craft.input_annotations:
    if annotation.kind == AnnotationKind.VOLATILE:
        outputs['volatiles'].append(annotation.name)
    elif annotation.kind == AnnotationKind.ARTEFACT:
        outputs['artifacts'].append(annotation.name)



['foo_out']

{
  "pipelines": {
    "fooer": [
      [
        {
          "module": "jupyter.foobar.craft_foo",
          "name": "craft_foo"
        }
      ]
    ],
    "spamer": [
      [
        {
          "module": "jupyter.spam.craft_spam",
          "name": "craft_spam"
        }
      ]
    ],
    "full": [
      [
        {
          "module": "jupyter.foobar.craft_foo",
          "name": "craft_foo"
        }
      ],
      [
        {
          "module": "jupyter.spam.craft_spam",
          "name": "craft_spam"
        }
      ]
    ]
  }
}
